In [95]:
import pandas as pd
from clickhouse_driver import Client
import os


In [96]:
lunch_duration = '23-29Nov2024'


In [97]:
# تنظیمات اتصال به ClickHouse
client = Client(
    host="172.21.16.1", 
    port="9000", 
    user='alireza_mohammadi', 
    password='4qNSwyNNlZ@Os0r8%#R7'
)
query = """
SELECT 
    code AS promoter_code,
    name_clear,
    cellphone_clear,
    city_id,
    m.state_en_3w as city,
    MAX(updated_at) AS updated_at
FROM 
    snapp_promoters.stream_users p
JOIN minio_db.operations_district_info m ON m.state_id = p.city_id
GROUP BY
    promoter_code, name_clear, cellphone_clear, city_id, m.state_en_3w
"""


In [98]:

clickhouse_data = client.execute(query)
clickhouse_df = pd.DataFrame(clickhouse_data, columns=['promoter_code', 'name_clear', 'cellphone_clear', 'city_id', 'city', 'updated_at'])


In [99]:

# خواندن فایل اکسل
df = pd.read_excel('E:/data/Data_Launch.xlsx', sheet_name='Final')
df_enriched = pd.read_excel('E:/data/Data_Launch.xlsx', sheet_name='Enriched')
quality_list = pd.read_excel('E:/data/Data_Launch.xlsx', sheet_name='quality_list')
df['promoter_code'] = df['promoter_code'].astype(str).str.strip()
clickhouse_df['promoter_code'] = clickhouse_df['promoter_code'].astype(int)


In [100]:

df=df[df['Eligibility'] != 'MayBeNextWeek']
df=df[df['Role'] == 'ON_SPOT_PROMOTER']


In [101]:
df.promoter_code = pd.to_numeric(df.promoter_code , errors='ignore')

C:\Users\Snapp\AppData\Local\Temp\ipykernel_22896\3608582873.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df.promoter_code = pd.to_numeric(df.promoter_code , errors='ignore')


In [102]:
# مرحله 1: فیلتر کردن پروموترهای Eligibility شده
filtered_df = df[df['Eligibility'] == 'Eligible']

# مرحله 2: ترکیب داده‌ها با اطلاعات شیت Enriched
enriched_filtered_df = filtered_df.merge(
    df_enriched[['promoter_code', 'quality_trained_place']],  # انتخاب ستون‌های مرتبط از شیت Enriched
    on='promoter_code',
    how='left'
)


In [103]:

# مرحله 3: انتخاب ردیف‌های با بهترین کیفیت (فرض می‌کنیم quality_police امتیازی است و باید بیشینه باشد)
enriched_filtered_df = enriched_filtered_df.sort_values(by='quality_trained_place', ascending=False).drop_duplicates(subset=['promoter_code'])


In [104]:

# پرومتر کدهای خاص که باید منتقل شوند
blacklist_promoter_codes = [
    24511, 76813, 28312, 35798, 25466, 24397, 18196, 95653, 71160, 87238,
    12719, 73838, 73749, 32655, 79781, 25519, 91761, 90583, 80222, 73473,
    50772, 92552, 76175, 46578, 38237, 81568, 52661, 58435, 84085, 51762,
    89915, 65323, 54952, 92874, 81061, 91610, 82608, 10014, 78810, 87774,
    80356, 65853, 40313, 31850, 42369, 61433, 50118, 71667, 92190, 77212,
    26822, 40423, 34828, 69728, 42939, 20561, 93202, 33154, 14550, 46388,
    42011, 96041, 31642, 90159, 92431, 93565, 67082, 27509, 97752, 36436,
    89334, 46586, 55424, 51911, 95519, 90601, 24752, 18601, 11188, 49292,
    66086, 85606, 51623, 59435, 10768, 25713, 75829, 59149, 96563, 36158,
    95684, 46657, 78477, 34001, 77236, 35163, 91819, 11722, 85601, 84036,
    60661, 93929, 71269, 96533, 65024, 49796, 54447, 87329, 15080, 75592,
    69644, 30750, 48503, 50954, 60099, 80389, 92740, 73896, 19386, 57196,
    34851, 61519, 60446, 92939, 63152, 16951, 78258, 64295, 11646, 93644
]


In [105]:
clickhouse_df.promoter_code = pd.to_numeric(clickhouse_df.promoter_code , errors='ignore')

C:\Users\Snapp\AppData\Local\Temp\ipykernel_22896\3600487540.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  clickhouse_df.promoter_code = pd.to_numeric(clickhouse_df.promoter_code , errors='ignore')


In [106]:

# حذف رکوردهای تکراری در clickhouse_df بر اساس 'promoter_code'
clickhouse_df = clickhouse_df.drop_duplicates(subset='promoter_code')

# # حذف پیش‌شماره +98 از شماره‌های موبایل
clickhouse_df['cellphone_clear'] = clickhouse_df['cellphone_clear'].str.replace(r'^\+98', '', regex=True)


# ادغام دیتافریم‌ها
enriched_filtered_df2 = pd.merge(enriched_filtered_df, clickhouse_df[['promoter_code', 'name_clear', 'cellphone_clear','city']].drop_duplicates(subset='promoter_code'), on='promoter_code', how='left')




In [107]:


# انتخاب ستون‌های نهایی
columns_to_save = ['promoter_code', 'City', 'name_clear', 'cellphone_clear' , 'quality_trained_place']  # اضافه کردن نام به ستون‌ها
merged_df = enriched_filtered_df2[columns_to_save]


In [108]:


blacklisted_df = merged_df[merged_df['promoter_code'].isin(blacklist_promoter_codes)]
blacklisted_df['Week'] = f'{lunch_duration}'


not_black_listed = merged_df[~merged_df['promoter_code'].isin(blacklist_promoter_codes)]
#فیلتر کردن رکوردهایی که شامل کوالیتی‌های خاص هستند
payment_df = not_black_listed[not_black_listed['quality_trained_place'].isin(quality_list['quality_trained_place'].tolist())]
payment_df['Rial'] = 5600000
payment_df['Description'] = f'Lunch incentive {lunch_duration}'


# حذف کوالیتی‌های خاص از دیتافریم اصلی
winner_df = not_black_listed[~not_black_listed['quality_trained_place'].isin(quality_list['quality_trained_place'].tolist())]
# حذف سطرهایی که کلمه 'پروموتر' در ستون 'name_clear' یا کلمات خاص در ستون 'City' وجود دارد
winner_df = winner_df[~winner_df['name_clear'].str.contains('پروموتر', case=False, na=False)]
winner_df = winner_df[~winner_df['City'].str.contains('QES', case=False, na=False)]
winner_df = winner_df[~winner_df['City'].str.contains('KIS', case=False, na=False)]

# حذف سطرهایی که مقدار promoter_code آن‌ها -1 است
winner_df = winner_df[winner_df['promoter_code'] != '-1']
columns_main_data = ['promoter_code', 'City']
columns_winner_data = ['cellphone_clear', 'City' ,'promoter_code']
columns_payment_data = ['cellphone_clear', 'City' ,'promoter_code' , 'Rial' , 'Description']
columns_blacklist_data = ['Week' , 'cellphone_clear', 'City' ,'promoter_code']
columns_Blacklisted_Promoters = ['promoter_code', 'City', 'quality_trained_place']




C:\Users\Snapp\AppData\Local\Temp\ipykernel_22896\1210167359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blacklisted_df['Week'] = f'{lunch_duration}'
C:\Users\Snapp\AppData\Local\Temp\ipykernel_22896\1210167359.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_df['Rial'] = 5600000
C:\Users\Snapp\AppData\Local\Temp\ipykernel_22896\1210167359.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [109]:
import pandas as pd

# مسیر ذخیره‌سازی فایل Excel
output_file = 'E:/data/output_file.xlsx'

# ایجاد فایل Excel با شیت‌های مختلف
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # ذخیره دیتافریم‌ها در شیت‌های مختلف
    winner_df.to_excel(writer, index=False, sheet_name='Winners')
    payment_df.to_excel(writer, index=False, sheet_name='Payments')
    blacklisted_df.to_excel(writer, index=False, sheet_name='Blacklisted')

print(f"Data has been saved to {output_file}")


Data has been saved to E:/data/output_file.xlsx
